In [1]:
# read data: 3 study groups by sub_list 
import nipype.interfaces.io as nio
import os
GROUPS=['PD','ET','NC']
OUT_DIR='/output/PD_ICA/'
SUB_LIST=[]; AGE_LIST=[]; JCOB_LIST=[];

for group_name in GROUPS:
    current_group=group_name
    current_sub_list_file = '/codes/devel/PD_Marker/'+current_group+'_info_ICA.list'
    # create dir for output
    current_OUT_DIR=OUT_DIR+current_group+'/'
    if not os.path.exists(current_OUT_DIR):
        os.makedirs(current_OUT_DIR)
    #read sub list
    with open(current_sub_list_file, 'r') as f_sub:
        sub_list_raw= f_sub.readlines()
    sub_list = [x[0:-1].split('\t')[0] for x in sub_list_raw] # remove 
    age_list = [int(x[0:-1].split('\t')[1]) for x in sub_list_raw]
    SUB_LIST.append(sub_list);  AGE_LIST.append(age_list);
    N_sub=len(sub_list)
    print(group_name, ': ', N_sub, sub_list, age_list)
    # grab group Jacobians
    ds_jacobian = nio.DataGrabber(infields=['sub_id'])
    ds_jacobian.inputs.base_directory = current_OUT_DIR # database
    ds_jacobian.inputs.template = '%s_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian.nii.gz' # from cwd
    ds_jacobian.inputs.sort_filelist = True
    ds_jacobian.inputs.sub_id = sub_list
    res_jacobian = ds_jacobian.run()
    jacobian_list=res_jacobian.outputs.outfiles
    JCOB_LIST.append(jacobian_list)
pd_sub_list = SUB_LIST[0]; et_sub_list = SUB_LIST[1]; nc_sub_list = SUB_LIST[2]; 
pd_age_list = AGE_LIST[0]; et_age_list = AGE_LIST[1]; nc_age_list = AGE_LIST[2];
pd_jaco_list=JCOB_LIST[0]; et_jaco_list=JCOB_LIST[1]; nc_jaco_list=JCOB_LIST[2];

PD :  40 ['sub-0002', 'sub-0004', 'sub-0005', 'sub-0006', 'sub-0008', 'sub-0009', 'sub-0012', 'sub-0014', 'sub-0015', 'sub-0021', 'sub-0022', 'sub-0023', 'sub-0024', 'sub-0025', 'sub-0028', 'sub-0030', 'sub-0031', 'sub-0034', 'sub-0035', 'sub-0037', 'sub-0038', 'sub-0040', 'sub-0047', 'sub-0051', 'sub-0052', 'sub-0068', 'sub-0075', 'sub-0076', 'sub-0094', 'sub-0096', 'sub-0098', 'sub-0109', 'sub-0111', 'sub-0118', 'sub-0125', 'sub-0129', 'sub-0132', 'sub-0136', 'sub-1000', 'sub-1020'] [70, 76, 45, 63, 59, 57, 47, 66, 65, 53, 62, 50, 63, 44, 61, 72, 51, 70, 69, 68, 54, 63, 75, 54, 56, 66, 66, 48, 66, 73, 57, 69, 69, 54, 60, 69, 74, 62, 77, 6]
ET :  30 ['sub-0016', 'sub-0061', 'sub-0081', 'sub-0115', 'sub-0119', 'sub-0122', 'sub-0134', 'sub-0178', 'sub-1012', 'sub-1120', 'sub-1160', 'sub-1230', 'sub-1310', 'sub-1340', 'sub-1450', 'sub-1500', 'sub-1690', 'sub-1890', 'sub-1920', 'sub-2400', 'sub-3600', 'sub-3700', 'sub-3900', 'sub-4200', 'sub-4300', 'sub-4700', 'sub-5700', 'sub-7000', 'sub

In [ ]:
# get mask
def mask_svJacobians(jacobian_in, mask_in, OUT_DIR):
    
# get data lists
from nipype.interfaces.ants import ANTS
from nipype.interfaces import fsl
import os,time
OUT_DIR='/output/PD_ICA/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)
atlas_9c='/templateflow/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c.nii'
atlas_9c_mask='/templateflow/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_mask.nii'
atlas_09_masked=OUT_DIR+'t1_MNI2009c_masked.nii.gz'
mask = fsl.ApplyMask(
    in_file=atlas_9c,
    out_file=atlas_09_masked,
    mask_file=atlas_9c_mask)
mask.run()

In [ ]:
# MELODIC ICA masked for ET vs HC
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage

index_PD=[0,10,18,19,20,21,22,23,25,-1] #vt11 missing
index_ET=[1,2,3, 4, 5, 6,7,8,9,11,12,13,14,15,16,17,24,34,36,38,39,40] #fmr07,20,28,33,47,78,108,
# 112,145, 189; VT1-3,41,,42,43,109 missing 33 missing from the list
index_HC=[26,27,28,29,30,31,32,33,35] #vt22 missing

t0=time.time()
g_path ='/output/sMRI/'
dir_all_jacobian = [g_path+t1_list[x][-29:-24]+'_t1_masked_Jacobian_masked.nii.gz' for x in index_ET+index_HC]
merged_file = g_path+'ICA_ETHC_masked/ETHC_masked_4d.nii.gz'

merger = fsl.Merge()
merger.inputs.in_files = dir_all_jacobian
merger.inputs.dimension = 'a'
merger.inputs.merged_file = merged_file
merger.cmdline
merger.run()
t1=time.time()
print('ET+HC Masked Jacobian:\n \tMegring 4D file takes: ', t1-t0)
melodic_setup = fsl.MELODIC()
melodic_setup.inputs.approach = 'tica'
melodic_setup.inputs.in_files = [merged_file]
#melodic_setup.inputs.no_bet = True
#melodic_setup.inputs.bg_threshold = 10
#melodic_setup.inputs.tr_sec = 1.5
#melodic_setup.inputs.mm_thresh = 0.5
#melodic_setup.inputs.out_stats = True
melodic_setup.inputs.out_all = True
melodic_setup.inputs.num_ICs = 30
#melodic_setup.inputs.t_des = 'timeDesign.mat'resICA
#melodic_setup.inputs.t_con = 'timeDesign.con'
#melodic_setup.inputs.s_des = 'subjectDesign.mat'
#melodic_setup.inputs.s_con = 'subjectDesign.con'
melodic_setup.inputs.out_dir = '/output/sMRI/ICA_ETHC_masked'
melodic_setup.inputs.report = True
melodic_setup.cmdline
melodic_setup.run()
t2=time.time()
print('\tET+HC MELODIC ICA takes: ', t2-t1)